# Classification Tree Program

In [1]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, adjusted_rand_score

class DecisionTree:
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1, criterion='entropy'):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.criterion = criterion
        self.tree = None                                                          # переменная, в которой будет храниться готовое дерево решений.
        self.feature_importances = None                                          # переменная для важности фич

    def entropy(self, y):
        counts = np.bincount(y)                                                   # Считаем количество объектов для каждого класса. Формат - [0,0,1,2,1,2,0]
        probabilities = counts / len(y)                                           # вероятность. Формат - [x/y, x1/y, x3/y]
        return -np.sum([p * np.log2(p) for p in probabilities if p > 0])          # суммируем вероятности. p - каждая итерация в полученном массиве 'probabilities'.

    def gini(self, y):
        counts = np.bincount(y)
        probabilities = counts / len(y)
        return 1 - np.sum(probabilities ** 2)

    def information_gain(self, y, left_indices, right_indices):
        if self.criterion == 'entropy':                                            # Выбор критерия
            impurity_func = self.entropy
        elif self.criterion == 'gini':
            impurity_func = self.gini
        else:
            raise ValueError(f"Unknown criterion: {self.criterion}")

        parent_impurity = impurity_func(y)                                         # неопределенность для всей выборки.
        left_impurity = impurity_func(y[left_indices])
        right_impurity = impurity_func(y[right_indices])

        n, n_left, n_right = len(y), len(left_indices), len(right_indices)
        weighted_impurity = (n_left / n) * left_impurity + (n_right / n) * right_impurity
        inf_gain = parent_impurity - weighted_impurity
        
        # print(f'Inf. gain "{self.criterion}": {inf_gain}')
        return inf_gain                                                            # возвращаем инф. выиг.
    
    
    def custom_1(self, y_oh, left_indices, right_indices):
        N = y_oh.sum()

        left = y_oh[left_indices]
        right = y_oh[right_indices]
        p_1 = left.sum() / N
        p_2 = right.sum() / N
        num_classes = y_oh.shape[1]                                                # .shape[1] кол-во столбцов, .shape[0] - кол-во строк.

        sum_total = 0
        
        for l in range(num_classes):
            p_1l = left[:, l].sum() / N
            p_2l = right[:, l].sum() / N
            p_l = p_1l + p_2l
            b = 1
            
            # Избегаем деления на ноль
            # if p_1 > 0:
            sum_total += ((p_1l - p_1 * p_l)**2) / p_1 * b**2
            # if p_2 > 0:
            sum_total += ((p_2l - p_2 * p_l)**2) / p_2 * b**2

        return N * sum_total
    
    
    def custom_2(self, y_oh, left_indices, right_indices):
        N = y_oh.sum()

        left = y_oh[left_indices]
        right = y_oh[right_indices]
        p_1 = left.sum() / N
        p_2 = right.sum() / N
        num_classes = y_oh.shape[1]                                                # .shape[1] кол-во столбцов, .shape[0] - кол-во строк.

        sum_total = 0
        
        for l in range(num_classes):
            p_1l = left[:, l].sum() / N
            p_2l = right[:, l].sum() / N
            p_l = p_1l + p_2l
            
            b = np.sqrt(p_l)
            
            # Избегаем деления на ноль
            # if p_1 > 0:
            sum_total += ((p_1l - p_1 * p_l)**2) / p_1 * b**2
            # if p_2 > 0:
            sum_total += ((p_2l - p_2 * p_l)**2) / p_2 * b**2

        return N * sum_total
    

    def custom_3(self, y_oh, left_indices, right_indices):
        N = y_oh.sum()

        left = y_oh[left_indices]
        right = y_oh[right_indices]
        p_1 = left.sum() / N
        p_2 = right.sum() / N
        num_classes = y_oh.shape[1]                                                # .shape[1] кол-во столбцов, .shape[0] - кол-во строк.

        sum_total = 0
        epsilon = 1e-10 
        
        for l in range(num_classes):
            p_1l = left[:, l].sum() / N
            p_2l = right[:, l].sum() / N
            p_l = p_1l + p_2l
            
            b = np.sqrt(p_l*(1 - p_l))
            
            # eps. для стабильности вычислений
            denominator_1 = max(p_1 * b**2, epsilon)
            denominator_2 = max(p_2 * b**2, epsilon)
            
            sum_total += ((p_1l - p_1 * p_l)**2) / denominator_1
            sum_total += ((p_2l - p_2 * p_l)**2) / denominator_2

        return N * sum_total
    

    def custom_4(self, y_oh, left_indices, right_indices):
        N = y_oh.sum()

        left = y_oh[left_indices]
        right = y_oh[right_indices]
        p_1 = left.sum() / N
        p_2 = right.sum() / N
        num_classes = y_oh.shape[1]                                                # .shape[1] кол-во столбцов, .shape[0] - кол-во строк.

        sum_total = 0
        
        for l in range(num_classes):
            p_1l = left[:, l].sum() / N
            p_2l = right[:, l].sum() / N
            p_l = p_1l + p_2l
            
            b = p_l
            
            sum_total += ((p_1l - p_1 * p_l)**2) / p_1 * b**2
            sum_total += ((p_2l - p_2 * p_l)**2) / p_2 * b**2


        return N * sum_total
    
    
    def custom_5(self, y_oh, left_indices, right_indices):
        N = y_oh.sum()

        left = y_oh[left_indices]
        right = y_oh[right_indices]
        p_1 = left.sum() / N
        p_2 = right.sum() / N
        num_classes = y_oh.shape[1]                                                # .shape[1] кол-во столбцов, .shape[0] - кол-во строк.

        sum_total = 0
        
        for l in range(num_classes):
            p_1l = left[:, l].sum() / N
            p_2l = right[:, l].sum() / N
            p_l = p_1l + p_2l
            
            b = p_l**2
            
            sum_total += ((p_1l - p_1 * p_l)**2) / p_1 * b**2
            sum_total += ((p_2l - p_2 * p_l)**2) / p_2 * b**2

        return N * sum_total
    

    def custom_6(self, y_oh, left_indices, right_indices):
        N = y_oh.sum()

        left = y_oh[left_indices]
        right = y_oh[right_indices]
        p_1 = left.sum() / N
        p_2 = right.sum() / N
        num_classes = y_oh.shape[1]                                                # .shape[1] кол-во столбцов, .shape[0] - кол-во строк.

        sum_total = 0
        epsilon = 1e-10
        
        for l in range(num_classes):
            p_1l = left[:, l].sum() / N
            p_2l = right[:, l].sum() / N
            p_l = p_1l + p_2l
            
            b = -np.log(max(p_l, epsilon))
            
            sum_total += ((p_1l - p_1 * p_l)**2) / p_1 * b**2
            sum_total += ((p_2l - p_2 * p_l)**2) / p_2 * b**2

        return N * sum_total
    

    def custom_7(self, y_oh, left_indices, right_indices):
        N = y_oh.sum()

        left = y_oh[left_indices]
        right = y_oh[right_indices]
        p_1 = left.sum() / N
        p_2 = right.sum() / N
        num_classes = y_oh.shape[1]                                                # .shape[1] кол-во столбцов, .shape[0] - кол-во строк.

        sum_total = 0
        epsilon = 1e-10
        
        for l in range(num_classes):
            p_1l = left[:, l].sum() / N
            p_2l = right[:, l].sum() / N
            p_l = p_1l + p_2l
            
            b = (-p_l)*np.log(max(p_l, epsilon))
            
            # # eps. для стабильности вычислений
            # denominator_1 = max(p_1 * b**2, epsilon)
            # denominator_2 = max(p_2 * b**2, epsilon)
            
            sum_total += ((p_1l - p_1 * p_l)**2) / p_1 * b**2
            sum_total += ((p_2l - p_2 * p_l)**2) / p_2 * b**2

        return N * sum_total
    
    
    def custom_8(self, y_oh, left_indices, right_indices):
        N = y_oh.sum()

        left = y_oh[left_indices]
        right = y_oh[right_indices]
        p_1 = left.sum() / N
        p_2 = right.sum() / N
        num_classes = y_oh.shape[1]                                                # .shape[1] кол-во столбцов, .shape[0] - кол-во строк.

        sum_total = 0
        epsilon = 1e-10
        
        for l in range(num_classes):
            p_1l = left[:, l].sum() / N
            p_2l = right[:, l].sum() / N
            p_l = p_1l + p_2l
            
            b = -(p_l**0.5) * np.log(max(p_l, epsilon))
            
            # # eps. для стабильности вычислений
            # denominator_1 = max(p_1 * b**2, epsilon)
            # denominator_2 = max(p_2 * b**2, epsilon)
            
            sum_total += ((p_1l - p_1 * p_l)**2) / p_1 * b**2
            sum_total += ((p_2l - p_2 * p_l)**2) / p_2 * b**2

        return N * sum_total    
    
    # Функция находит наиболее частый элемент в массиве y (метки классов).
    def most_common_label(self, y):
        return Counter(y).most_common(1)[0][0]


    def find_best_split(self, X, y, num_features, y_oh=None):
        best_gain = -float('inf')                                                  # хранит лучшее значение критерия
        best_split = None                                                          # будет содержать параметры наилучшего разбиения

        for feature_index in range(num_features):                                  # перебираем по очереди признаки.
            # Сортируем значения признака
            feature_values = np.sort(X[:, feature_index])
            # Берем средние между соседними значениями
            thresholds = (feature_values[:-1] + feature_values[1:]) / 2     
            
            for threshold in thresholds:                                           # для каждого уникального значения делим данные на 2 части.
                left_indices = np.where(X[:, feature_index] <= threshold)[0]       # левый - меньше уникального значения. [0] - нужен для возвращения массива, а не кортежа.
                right_indices = np.where(X[:, feature_index] > threshold)[0]       # правый - больше ун. знач. feature_index - искомый признак.

                if (len(left_indices) < self.min_samples_leaf or 
                    len(right_indices) < self.min_samples_leaf):
                    continue                                                       # если условие срабатывает, переходим к следующей итерации, пропуская то, что ниже.

                if self.criterion == 'custom_1':
                    if y_oh is None:
                        raise ValueError("y_oh required for custom_1 criterion")
                    gain = self.custom_1(y_oh, left_indices, right_indices)
                
                elif self.criterion == 'custom_2':
                    if y_oh is None:
                        raise ValueError("y_oh required for custom_2 criterion")
                    gain = self.custom_2(y_oh, left_indices, right_indices)
                
                elif self.criterion == 'custom_3':
                    if y_oh is None:
                        raise ValueError("y_oh required for custom_3 criterion")
                    gain = self.custom_3(y_oh, left_indices, right_indices)                    
                
                elif self.criterion == 'custom_4':
                    if y_oh is None:
                        raise ValueError("y_oh required for custom_4 criterion")
                    gain = self.custom_4(y_oh, left_indices, right_indices)
                
                elif self.criterion == 'custom_5':
                    if y_oh is None:
                        raise ValueError("y_oh required for custom_5 criterion")
                    gain = self.custom_5(y_oh, left_indices, right_indices)
                
                elif self.criterion == 'custom_6':
                    if y_oh is None:
                        raise ValueError("y_oh required for custom_6 criterion")
                    gain = self.custom_6(y_oh, left_indices, right_indices)    
                    
                elif self.criterion == 'custom_7':
                    if y_oh is None:
                        raise ValueError("y_oh required for custom_7 criterion")
                    gain = self.custom_7(y_oh, left_indices, right_indices)
                    
                elif self.criterion == 'custom_8':
                    if y_oh is None:
                        raise ValueError("y_oh required for custom_7 criterion")
                    gain = self.custom_8(y_oh, left_indices, right_indices)                      
                
                else:
                    gain = self.information_gain(y, left_indices, right_indices)   # рассчитываем инф. прирост.

                if gain > best_gain:                                               # если текущий прирост больше самого большого
                    best_gain = gain                                               # приравниваем переменную наибольшего к текущему.
                    best_split = {
                        'feature_index': feature_index,
                        'threshold': threshold,
                        'left_indices': left_indices,
                        'right_indices': right_indices,
                        'gain': gain
                    }                                                              # теперь это параметры разбиения, которые дают наилучший прирост.
        
        return best_split                                                          # После перебора всех признаков и порогов, возвращаем параметры лучшего найденного разбиения.


    def fit(self, X, y, y_oh=None):
        num_features = X.shape[1]
        self.feature_importances = np.zeros(num_features)                          # инициализируем нулями
        self.tree = self.grow_tree(X, y, y_oh, depth=0)

        # нормализуем важности, чтобы сумма = 1, как в sklearn
        total = self.feature_importances.sum()
        if total > 0:
            self.feature_importances /= total


    def grow_tree(self, X, y, y_oh, depth):
        num_samples, num_features = X.shape
        num_classes = len(set(y))

        if (depth == self.max_depth or 
            num_classes == 1 or 
            num_samples < self.min_samples_split):
            return self.most_common_label(y)

        if self.criterion.startswith('custom_'):
            best_split = self.find_best_split(X, y, num_features, y_oh)
        else:
            best_split = self.find_best_split(X, y, num_features)

        if best_split is None:
            return self.most_common_label(y)

        left_indices, right_indices = best_split['left_indices'], best_split['right_indices']
        
        # Вычисляем прирост информации для подсчета важности признаков
        if self.criterion == 'custom_1':
            gain = self.custom_1(y_oh, left_indices, right_indices)
        elif self.criterion == 'custom_2':
            gain = self.custom_2(y_oh, left_indices, right_indices)
        elif self.criterion == 'custom_3':
            gain = self.custom_3(y_oh, left_indices, right_indices)
        elif self.criterion == 'custom_4':
            gain = self.custom_4(y_oh, left_indices, right_indices)
        elif self.criterion == 'custom_5':
            gain = self.custom_5(y_oh, left_indices, right_indices)
        elif self.criterion == 'custom_6':
            gain = self.custom_6(y_oh, left_indices, right_indices)
        elif self.criterion == 'custom_7':
            gain = self.custom_7(y_oh, left_indices, right_indices)
        elif self.criterion == 'custom_8':
            gain = self.custom_8(y_oh, left_indices, right_indices)            
            
        else:
            gain = self.information_gain(y, left_indices, right_indices)

        self.feature_importances[best_split['feature_index']] += gain              # Сохраняем вклад этого признака в важность

        left_subtree = self.grow_tree(X[left_indices], y[left_indices], 
                                    y_oh[left_indices] if y_oh is not None else None, 
                                    depth + 1)
        right_subtree = self.grow_tree(X[right_indices], y[right_indices], 
                                     y_oh[right_indices] if y_oh is not None else None, 
                                     depth + 1)

        return {
            'feature_index': best_split['feature_index'],
            'threshold': best_split['threshold'],
            'left': left_subtree,
            'right': right_subtree
        }


    def predict(self, X):
        return np.array([self._traverse_tree(x, self.tree) for x in X])


    def _traverse_tree(self, x, node):
        if isinstance(node, dict):
            if x[node['feature_index']] <= node['threshold']:
                return self._traverse_tree(x, node['left'])
            else:
                return self._traverse_tree(x, node['right'])

        return node                                                             # Если нет, то это лист и присваиваем метку.

### 1 Experiment

In [99]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier


def compare_metrics_train_test(max_depth, X, y, *, N=None, V=None, k=None, alpha=None, nmin=None, random_state=42):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=random_state)

    encoder = OneHotEncoder(sparse_output=False)
    y_oh_train = encoder.fit_transform(y_train.reshape(-1,1))


    '''Custom_1'''
    custom_1 = DecisionTree(max_depth=max_depth, criterion='custom_1')
    custom_1.fit(X_train, y_train, y_oh_train)
    y_pred = custom_1.predict(X_test)
    accuracy_1, precision_1 = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall_1, f1_1 = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
    ari_1 = adjusted_rand_score(y_test, y_pred)

    '''GINI'''
    gini = DecisionTree(max_depth=max_depth, criterion='gini')
    gini.fit(X_train, y_train)
    y_pred = gini.predict(X_test)
    accuracy_gini, precision_gini = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall_gini, f1_gini = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
    ari_gini = adjusted_rand_score(y_test, y_pred)

    '''Sklearn_GINI'''
    sk_gini = DecisionTreeClassifier(max_depth=max_depth, criterion='gini')
    sk_gini.fit(X_train, y_train)
    y_pred = sk_gini.predict(X_test)
    accuracy_gini_sk, precision_gini_sk = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall_gini_sk, f1_gini_sk = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
    ari_gini_sk = adjusted_rand_score(y_test, y_pred)

    '''Entropy'''
    entropy = DecisionTree(max_depth=max_depth, criterion='entropy')
    entropy.fit(X_train, y_train)
    y_pred = entropy.predict(X_test)
    accuracy_entropy, precision_entropy = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall_entropy, f1_entropy = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
    ari_entropy = adjusted_rand_score(y_test, y_pred)

    '''Sklearn_Entropy'''
    sk_entropy = DecisionTreeClassifier(max_depth=max_depth, criterion='entropy')
    sk_entropy.fit(X_train, y_train)
    y_pred = sk_entropy.predict(X_test)
    accuracy_entropy_sk, precision_entropy_sk = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall_entropy_sk, f1_entropy_sk = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
    ari_entropy_sk = adjusted_rand_score(y_test, y_pred)
    
    '''Custom_2'''
    custom_2 = DecisionTree(max_depth=max_depth, criterion='custom_2')
    custom_2.fit(X_train, y_train, y_oh_train)
    y_pred = custom_2.predict(X_test)
    accuracy_2, precision_2 = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall_2, f1_2 = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
    ari_2 = adjusted_rand_score(y_test, y_pred)

    '''Custom_3'''
    custom_3 = DecisionTree(max_depth=max_depth, criterion='custom_3')
    custom_3.fit(X_train, y_train, y_oh_train)
    y_pred = custom_3.predict(X_test)
    accuracy_3, precision_3 = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall_3, f1_3 = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
    ari_3 = adjusted_rand_score(y_test, y_pred)

    '''Custom_4'''
    custom_4 = DecisionTree(max_depth=max_depth, criterion='custom_4')
    custom_4.fit(X_train, y_train, y_oh_train)
    y_pred = custom_4.predict(X_test)
    accuracy_4, precision_4 = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall_4, f1_4 = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
    ari_4 = adjusted_rand_score(y_test, y_pred)

    '''Custom_5'''
    custom_5 = DecisionTree(max_depth=max_depth, criterion='custom_5')
    custom_5.fit(X_train, y_train, y_oh_train)
    y_pred = custom_5.predict(X_test)
    accuracy_5, precision_5 = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall_5, f1_5 = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
    ari_5 = adjusted_rand_score(y_test, y_pred)

    '''Custom_6'''
    custom_6 = DecisionTree(max_depth=max_depth, criterion='custom_6')
    custom_6.fit(X_train, y_train, y_oh_train)
    y_pred = custom_6.predict(X_test)
    accuracy_6, precision_6 = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall_6, f1_6 = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
    ari_6 = adjusted_rand_score(y_test, y_pred)

    '''Custom_7'''
    custom_7 = DecisionTree(max_depth=max_depth, criterion='custom_7')
    custom_7.fit(X_train, y_train, y_oh_train)
    y_pred = custom_7.predict(X_test)
    accuracy_7, precision_7 = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall_7, f1_7 = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
    ari_7 = adjusted_rand_score(y_test, y_pred)
    
    '''Custom_8'''
    custom_8 = DecisionTree(max_depth=max_depth, criterion='custom_8')
    custom_8.fit(X_train, y_train, y_oh_train)
    y_pred = custom_8.predict(X_test)
    accuracy_8, precision_8 = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall_8, f1_8 = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
    ari_8 = adjusted_rand_score(y_test, y_pred)

    results = np.round([[accuracy_1, accuracy_gini, accuracy_gini_sk, accuracy_entropy, accuracy_entropy_sk, accuracy_2, accuracy_3, accuracy_4, accuracy_5, accuracy_6, accuracy_7, accuracy_8],
                    [precision_1, precision_gini, precision_gini_sk, precision_entropy, precision_entropy_sk, precision_2, precision_3, precision_4, precision_5, precision_6, precision_7, precision_8],
                    [recall_1, recall_gini, recall_gini_sk, recall_entropy, recall_entropy_sk, recall_2, recall_3, recall_4, recall_5, recall_6, recall_7, recall_8],
                    [f1_1, f1_gini, f1_gini_sk, f1_entropy, f1_entropy_sk, f1_2, f1_3, f1_4, f1_5, f1_6, f1_7, f1_8],
                    [ari_1, ari_gini, ari_gini_sk, ari_entropy, ari_entropy_sk, ari_2, ari_3, ari_4, ari_5, ari_6, ari_7, ari_8],],4)

    column = ['b = 1','gini','gini_sklearn', 'entropy', 'entropy_sklearn', 'b = p_l ^ 0.5', 'b = (p_l*(1 - p_l)) ^ 0.5', 'b = p_l', 'b = p_l ^ 2', 'b = log(p_l)', 'b = -p_l * log(p_l)', 'b = p_l^0.5 * log(p_l)']
    table = pd.DataFrame(data=results, columns=column, index=['Accuracy', 'Precision', 'Recall','F1 score','ARI'])
    
    print(f'\nN, V, k, alpha, nmin, max_depth = {N, V, k, alpha, nmin, max_depth}')

    return table

### Mean/std of 50 exps.

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier


def compare_metrics_train_test(max_depth, X, y, *, N=None, V=None, k=None, alpha=None, nmin=None):
    
    all_results = []
    
    for seed in range(1,51):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed)

        encoder = OneHotEncoder(sparse_output=False)
        y_oh_train = encoder.fit_transform(y_train.reshape(-1,1))


        '''Custom_1'''
        custom_1 = DecisionTree(max_depth=max_depth, criterion='custom_1')
        custom_1.fit(X_train, y_train, y_oh_train)
        y_pred = custom_1.predict(X_test)
        accuracy_1, precision_1 = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
        recall_1, f1_1 = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
        ari_1 = adjusted_rand_score(y_test, y_pred)

        # '''GINI'''
        # gini = DecisionTree(max_depth=max_depth, criterion='gini')
        # gini.fit(X_train, y_train)
        # y_pred = gini.predict(X_test)
        # accuracy_gini, precision_gini = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
        # recall_gini, f1_gini = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
        # ari_gini = adjusted_rand_score(y_test, y_pred)

        # '''Sklearn_GINI'''
        # sk_gini = DecisionTreeClassifier(max_depth=max_depth, criterion='gini')
        # sk_gini.fit(X_train, y_train)
        # y_pred = sk_gini.predict(X_test)
        # accuracy_gini_sk, precision_gini_sk = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
        # recall_gini_sk, f1_gini_sk = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
        # ari_gini_sk = adjusted_rand_score(y_test, y_pred)

        # '''Entropy'''
        # entropy = DecisionTree(max_depth=max_depth, criterion='entropy')
        # entropy.fit(X_train, y_train)
        # y_pred = entropy.predict(X_test)
        # accuracy_entropy, precision_entropy = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
        # recall_entropy, f1_entropy = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
        # ari_entropy = adjusted_rand_score(y_test, y_pred)

        '''Sklearn_Entropy'''
        sk_entropy = DecisionTreeClassifier(max_depth=max_depth, criterion='entropy')
        sk_entropy.fit(X_train, y_train)
        y_pred = sk_entropy.predict(X_test)
        accuracy_entropy_sk, precision_entropy_sk = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
        recall_entropy_sk, f1_entropy_sk = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
        ari_entropy_sk = adjusted_rand_score(y_test, y_pred)
        
        '''Custom_2'''
        custom_2 = DecisionTree(max_depth=max_depth, criterion='custom_2')
        custom_2.fit(X_train, y_train, y_oh_train)
        y_pred = custom_2.predict(X_test)
        accuracy_2, precision_2 = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
        recall_2, f1_2 = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
        ari_2 = adjusted_rand_score(y_test, y_pred)

        '''Custom_3'''
        custom_3 = DecisionTree(max_depth=max_depth, criterion='custom_3')
        custom_3.fit(X_train, y_train, y_oh_train)
        y_pred = custom_3.predict(X_test)
        accuracy_3, precision_3 = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
        recall_3, f1_3 = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
        ari_3 = adjusted_rand_score(y_test, y_pred)

        '''Custom_4'''
        custom_4 = DecisionTree(max_depth=max_depth, criterion='custom_4')
        custom_4.fit(X_train, y_train, y_oh_train)
        y_pred = custom_4.predict(X_test)
        accuracy_4, precision_4 = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
        recall_4, f1_4 = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
        ari_4 = adjusted_rand_score(y_test, y_pred)

        '''Custom_5'''
        custom_5 = DecisionTree(max_depth=max_depth, criterion='custom_5')
        custom_5.fit(X_train, y_train, y_oh_train)
        y_pred = custom_5.predict(X_test)
        accuracy_5, precision_5 = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
        recall_5, f1_5 = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
        ari_5 = adjusted_rand_score(y_test, y_pred)

        '''Custom_6'''
        custom_6 = DecisionTree(max_depth=max_depth, criterion='custom_6')
        custom_6.fit(X_train, y_train, y_oh_train)
        y_pred = custom_6.predict(X_test)
        accuracy_6, precision_6 = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
        recall_6, f1_6 = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
        ari_6 = adjusted_rand_score(y_test, y_pred)

        '''Custom_7'''
        custom_7 = DecisionTree(max_depth=max_depth, criterion='custom_7')
        custom_7.fit(X_train, y_train, y_oh_train)
        y_pred = custom_7.predict(X_test)
        accuracy_7, precision_7 = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
        recall_7, f1_7 = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
        ari_7 = adjusted_rand_score(y_test, y_pred)


        '''Custom_8'''
        custom_8 = DecisionTree(max_depth=max_depth, criterion='custom_8')
        custom_8.fit(X_train, y_train, y_oh_train)
        y_pred = custom_8.predict(X_test)
        accuracy_8, precision_8 = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='weighted', zero_division=0)
        recall_8, f1_8 = recall_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='weighted')
        ari_8 = adjusted_rand_score(y_test, y_pred)

        results = np.round([[accuracy_1, accuracy_entropy_sk, accuracy_2, accuracy_3, accuracy_4, accuracy_5, accuracy_6, accuracy_7, accuracy_8],
                        [precision_1, precision_entropy_sk, precision_2, precision_3, precision_4, precision_5, precision_6, precision_7, precision_8],
                        [recall_1, recall_entropy_sk, recall_2, recall_3, recall_4, recall_5, recall_6, recall_7, recall_8],
                        [f1_1, f1_entropy_sk, f1_2, f1_3, f1_4, f1_5, f1_6, f1_7, f1_8],
                        [ari_1, ari_entropy_sk, ari_2, ari_3, ari_4, ari_5, ari_6, ari_7, ari_8],],4)
        
        all_results.append(results)
        print(f'Finished: {seed} iter.')
        
    print(f'\nN, V, k, alpha, nmin, max_depth = {N, V, k, alpha, nmin, max_depth}')
            
    all_results = np.array(all_results)  # shape: (4, 5, 11)

    mean_results = np.round(np.mean(all_results, axis=0),4)
    std_results = np.round(np.std(all_results, axis=0),4)
    
    # Final table (Mean/std)
    columns = ['b = 1','entropy_sklearn', 'b = p_l ^ 0.5', 'b = (p_l*(1 - p_l)) ^ 0.5', 'b = p_l', 'b = p_l ^ 2', 'b = -log(p_l)', 'b = -p_l * log(p_l)', 'b = -p_l^0.5 * log(p_l)']
    metrics = ['Accuracy', 'Precision', 'Recall', 'F1 score', 'ARI']
    
    index_tuples = []
    for metric in metrics:
        index_tuples.append((metric, 'Mean'))
        index_tuples.append((metric, 'Std'))
    
    multi_index = pd.MultiIndex.from_tuples(index_tuples, names=['Metric', 'Statistic'])
    
    # Final table
    final_table_data = []
    for i in range(len(metrics)):
        final_table_data.append(mean_results[i])
        final_table_data.append(std_results[i])
    
    final_table = pd.DataFrame(final_table_data, 
                             columns=columns, 
                             index=multi_index)
    
    return final_table